# Introduction

Welcome to this tutorial on using jupyter notebooks with Moveit 2. A great benefit of being able to interact with MoveIt via a Python notebook is the ability to rapidly prototype code. We hope you find this interface intuitive and that you gain value from using MoveIt via Python notebooks.

In this tutorial we will cover the following: 

* The required imports to run the notebook
* A motion planning example
* A teleoperation example

If you have suggestions or feedback for this tutorial please post an issue on GitHub (https://github.com/ros-planning/moveit2_tutorials) and tag @peterdavidfagan.

## Imports

Note: to launch this notebook and the nodes it depends on you must first specify a launch file. Details are provided earlier in this tutorial ().

In [1]:
import os
import math
import sys
import yaml
import rclpy
import numpy as np

# message libraries
from geometry_msgs.msg import PoseStamped, Pose

# moveit_py
from moveit.planning import MoveItPy
from moveit.core.robot_state import RobotState
from moveit.core.kinematic_constraints import construct_joint_constraint

# config file libraries
from moveit_configs_utils import MoveItConfigsBuilder
from ament_index_python.packages import get_package_share_directory


# we need to specify our moveit_py config at the top of each notebook we use. 
# this is since we will start spinning a moveit_py node within this notebook.

moveit_config = (
        MoveItConfigsBuilder(robot_name="UF_ROBOT", package_name="lite6_enrico")
        .robot_description_semantic(file_path="config/UF_ROBOT.srdf")
        .trajectory_execution(file_path="config/moveit_controllers.yaml")
        .robot_description(file_path="config/UF_ROBOT.urdf.xacro")
        .moveit_cpp(
            file_path=get_package_share_directory("lite6_moveit_demos")
            + "/config/moveit_cpp.yaml"
        )
        .to_moveit_configs()
    ).to_dict()


## Setup

In [2]:
# initialise rclpy (only for logging purposes)
#rclpy.init()

# instantiate moveit_py instance and a planning component for the panda_arm
moveit = MoveItPy(node_name="moveit_py", config_dict=moveit_config)
lite6 = moveit.get_planning_component("lite6_arm")

[INFO] [1713539298.636103333] [moveit_3122146238.moveit.py.cpp_initializer]: Initialize rclcpp
[INFO] [1713539298.636162218] [moveit_3122146238.moveit.py.cpp_initializer]: Initialize node parameters
[INFO] [1713539298.636177481] [moveit_3122146238.moveit.py.cpp_initializer]: Initialize node and executor
[INFO] [1713539298.655956718] [moveit_3122146238.moveit.py.cpp_initializer]: Spin separate thread
[INFO] [1713539298.695991909] [moveit_3122146238.moveit.ros.rdf_loader]: Loaded robot model in 0.00543863 seconds
[INFO] [1713539298.696037436] [moveit_3122146238.moveit.core.robot_model]: Loading robot model 'UF_ROBOT'...
[INFO] [1713539298.696050706] [moveit_3122146238.moveit.core.robot_model]: No root/virtual joint specified in SRDF. Assuming fixed joint
[INFO] [1713539298.737673573] [moveit_3122146238.moveit.kinematics.kdl_kinematics_plugin]: Joint weights for group 'lite6_arm': 1 1 1 1 1 1
[INFO] [1713539298.826958235] [moveit_3122146238.moveit.ros.planning_scene_monitor]: Publishing m

## Motion Planning Example

In [3]:
lite6.set_start_state_to_current_state()

# set constraints message
joint_values = {
    "joint1": math.radians(0),
    "joint2": math.radians(10.4),
    "joint3": math.radians(31.1),
    "joint4": math.radians(-1.5),
    "joint5": math.radians(21.5),
    "joint6": math.radians(1.3),
}
robot_state = RobotState(moveit.get_robot_model())
robot_state.joint_positions = joint_values
joint_constraint = construct_joint_constraint(
    robot_state=robot_state,
    joint_model_group=moveit.get_robot_model().get_joint_model_group("lite6_arm"),
)
lite6.set_goal_state(motion_plan_constraints=[joint_constraint])

# plan to goal
plan_result = lite6.plan()

# execute the plan
if plan_result:
    robot_trajectory = plan_result.trajectory
    moveit.execute(robot_trajectory, controllers=[])

[WARN] [1713539301.488820058] [moveit_py]: Parameter 'plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[INFO] [1713539301.489029549] [moveit_py]: Calling PlanningRequestAdapter 'ResolveConstraintFrames'
[INFO] [1713539301.489126229] [moveit_py]: Calling PlanningRequestAdapter 'ValidateWorkspaceBounds'
[WARN] [1713539301.489138898] [moveit_3122146238.moveit.ros.validate_workspace_bounds]: It looks like the planning volume was not specified. Using default values.
[INFO] [1713539301.489184621] [moveit_py]: Calling PlanningRequestAdapter 'CheckStartStateBounds'
[INFO] [1713539301.489241329] [moveit_py]: Calling PlanningRequestAdapter 'CheckStartStateCollision'
[INFO] [1713539301.489445032] [moveit_3122146238.moveit.planners.pilz.trajectory_generator.ptp]: Initialized Point-to-Point Trajectory Generator.
[INFO] [1713539301.489470185] [moveit_py]: Calling Planner 'Pilz Industrial Motion Planner'
[INFO] [17135393

In [4]:
lite6.set_start_state_to_current_state()
lite6.set_goal_state(configuration_name="Ready")
# plan to goal
plan_result = lite6.plan()

# execute the plan
if plan_result:
    robot_trajectory = plan_result.trajectory
    moveit.execute(robot_trajectory, controllers=[])

[WARN] [1713539306.902989220] [moveit_py]: Parameter 'plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[INFO] [1713539306.903275702] [moveit_py]: Calling PlanningRequestAdapter 'ResolveConstraintFrames'
[INFO] [1713539306.903457333] [moveit_py]: Calling PlanningRequestAdapter 'ValidateWorkspaceBounds'
[WARN] [1713539306.903495824] [moveit_3122146238.moveit.ros.validate_workspace_bounds]: It looks like the planning volume was not specified. Using default values.
[INFO] [1713539306.903565563] [moveit_py]: Calling PlanningRequestAdapter 'CheckStartStateBounds'
[INFO] [1713539306.903659362] [moveit_py]: Calling PlanningRequestAdapter 'CheckStartStateCollision'
[INFO] [1713539306.903796103] [moveit_3122146238.moveit.planners.pilz.trajectory_generator.ptp]: Initialized Point-to-Point Trajectory Generator.
[INFO] [1713539306.903815015] [moveit_py]: Calling Planner 'Pilz Industrial Motion Planner'
[INFO] [17135393

## Teleop